In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline
tf.__version__

'2.18.0'

In [2]:
Car_price = pd.read_csv(r"Datasets\CarPrice_Assignment.csv")
Car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
Car_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

Since Carname has morethan 50 no of categories we dropping that column

In [4]:
Car_price = Car_price.drop(columns=['car_ID', 'CarName'], axis=1)
Car_price.shape

(205, 24)

In [5]:
columns_to_be_encoded ={}
for column in Car_price.columns:
    if Car_price[column].dtype == 'object':
        columns_to_be_encoded[column] = Car_price[column].unique()
columns_to_be_encoded

{'fueltype': array(['gas', 'diesel'], dtype=object),
 'aspiration': array(['std', 'turbo'], dtype=object),
 'doornumber': array(['two', 'four'], dtype=object),
 'carbody': array(['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
       dtype=object),
 'drivewheel': array(['rwd', 'fwd', '4wd'], dtype=object),
 'enginelocation': array(['front', 'rear'], dtype=object),
 'enginetype': array(['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv'], dtype=object),
 'cylindernumber': array(['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
       dtype=object),
 'fuelsystem': array(['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi'],
       dtype=object)}

In [6]:
one_hot_columns = []
label_encode_columns = []
for column, unique_values in columns_to_be_encoded.items():
    if len(unique_values) < 4:
        one_hot_columns.append(column)
    else:
        label_encode_columns.append(column)
one_hot_columns, label_encode_columns

(['fueltype', 'aspiration', 'doornumber', 'drivewheel', 'enginelocation'],
 ['carbody', 'enginetype', 'cylindernumber', 'fuelsystem'])

In [7]:
for column in Car_price.columns:
    if Car_price[column].dtype != 'object':
        print(f"{column}: {Car_price[column].mean()}")
        print(f"{column}: {Car_price[column].median()}")

symboling: 0.8341463414634146
symboling: 1.0
wheelbase: 98.75658536585367
wheelbase: 97.0
carlength: 174.04926829268288
carlength: 173.2
carwidth: 65.90780487804878
carwidth: 65.5
carheight: 53.72487804878049
carheight: 54.1
curbweight: 2555.5658536585365
curbweight: 2414.0
enginesize: 126.90731707317073
enginesize: 120.0
boreratio: 3.329756097560975
boreratio: 3.31
stroke: 3.255414634146341
stroke: 3.29
compressionratio: 10.142536585365855
compressionratio: 9.0
horsepower: 104.1170731707317
horsepower: 95.0
peakrpm: 5125.121951219512
peakrpm: 5200.0
citympg: 25.21951219512195
citympg: 24.0
highwaympg: 30.75121951219512
highwaympg: 30.0
price: 13276.710570731706
price: 10295.0


In [8]:
columns_to_be_scaled = []
for column in Car_price.columns:
    if Car_price[column].dtype != 'object' and column != 'symboling' and column != 'price':
        columns_to_be_scaled.append(column)
columns_to_be_scaled

['wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginesize',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg']

In [15]:
from sklearn.preprocessing import LabelEncoder
Car_price_encoded = Car_price
le = LabelEncoder()
for column in label_encode_columns:
    Car_price_encoded[column] = le.fit_transform(Car_price_encoded[column])
    print(f"{column}: {Car_price_encoded[column].unique()}")
Car_price_encoded.head()

carbody: [0 2 3 4 1]
enginetype: [0 5 3 2 6 4 1]
cylindernumber: [2 3 1 4 5 6 0]
fuelsystem: [5 1 4 0 7 2 3 6]


,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,gas,std,two,0,rwd,front,88.6,168.8,64.1,...,130,5,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,gas,std,two,0,rwd,front,88.6,168.8,64.1,...,130,5,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,gas,std,two,2,rwd,front,94.5,171.2,65.5,...,152,5,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,gas,std,four,3,fwd,front,99.8,176.6,66.2,...,109,5,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,gas,std,four,3,4wd,front,99.4,176.6,66.4,...,136,5,3.19,3.40,8.0,115,5500,18,22,17450.0


In [16]:
X, y = Car_price_encoded.drop(columns=['price'], axis=1), Car_price['price']
X.shape, y.shape

((205, 23), (205,))

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   fueltype          205 non-null    object 
 2   aspiration        205 non-null    object 
 3   doornumber        205 non-null    object 
 4   carbody           205 non-null    int64  
 5   drivewheel        205 non-null    object 
 6   enginelocation    205 non-null    object 
 7   wheelbase         205 non-null    float64
 8   carlength         205 non-null    float64
 9   carwidth          205 non-null    float64
 10  carheight         205 non-null    float64
 11  curbweight        205 non-null    int64  
 12  enginetype        205 non-null    int64  
 13  cylindernumber    205 non-null    int64  
 14  enginesize        205 non-null    int64  
 15  fuelsystem        205 non-null    int64  
 16  boreratio         205 non-null    float64
 1

In [22]:
type(one_hot_columns)

list

In [ ]:
from sklearn.preprocessing import  OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
ct_ohe = make_column_transformer(
    (OneHotEncoder(sparse_output=False), one_hot_columns),
    (MinMaxScaler(), columns_to_be_scaled)
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ct_ohe.fit(X_train)
X_train_min_scaled = ct_ohe.transform(X_train)
X_test_min_scaled = ct_ohe.transform(X_test)
X_train.shape, X_train_min_scaled.shape, X_test_min_scaled.shape, y_train.shape, y_test.shape

((164, 23), (164, 24), (41, 24), (164,), (41,))

In [30]:
X_train_min_scaled[0]

array([1.        , 0.        , 1.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.63103448, 0.52413793, 0.40952381, 0.55      ,
       0.39207833, 0.25      , 0.63571429, 0.74761905, 0.9375    ,
       0.08474576, 0.02040816, 0.5625    , 0.67647059])

ValueError: A given column is not a column of the dataframe